In [ ]:
!pip install rasterio

In [68]:
from osgeo import gdal
import os,  re
from pathlib import Path
import rasterio
import rasterio.plot
import matplotlib.pyplot as plt
from pathlib import Path

In [69]:
# Root Directory
Image_Segmentation_Path = '/content/drive/My Drive/Image Segmentation/'

# Inputs/Sources
Processed_DEMs_Path = Image_Segmentation_Path + "Processed DEMs/"
high_dem_interpolated = Processed_DEMs_Path + "high_dem_interpolated.tif"
annotation_file = Processed_DEMs_Path + "annotated.tif"
slope_path = Processed_DEMs_Path + "slope.tif"
aspect_path = Processed_DEMs_Path + "aspect.tif"
hillshade_path = Processed_DEMs_Path + "hillshade.tif"

#Outputs/Destinations
Tiling_Path = Image_Segmentation_Path + "Tiles Sets/"
DEM_Tiles_Sets = Tiling_Path + "DEM Tiles Sets/"
Annotated_Tiles_Sets = Tiling_Path + "Annotated Tiles Sets/"
Slope_Tiles_Sets = Tiling_Path + "Slope Tiles Sets/"
Aspect_Tiles_Sets = Tiling_Path + "Aspect Tiles Sets/"
Hillshade_Tiles_Sets = Tiling_Path + "Hillshade Tiles Sets/"

# PNGs for the report 
PNG_Path = Image_Segmentation_Path + "PNG/"

In [70]:
# inputs = [high_dem_interpolated,
#           annotation_file,
#           slope_path,
#           aspect_path,
#           hillshade_path]

In [71]:
def get_tile_size(xlen, xdiv, ylen, ydiv):
  xsize = xlen/xdiv
  ysize = ylen/ydiv
  return xsize, ysize

In [96]:
def get_steps(xdiv, xmin, xsize, ydiv, ymax, ysize):
  xsteps = [xmin + xsize * i for i in range(xdiv+1)]
  ysteps = [ymax - ysize * i for i in range(ydiv+1)]
  return xsteps, ysteps

In [106]:
def tile(source_DEM, output_path, xsteps, ysteps, dstNodata):  
  for i in range(len(xsteps)-1):
    for j in range(len(ysteps)-1):
      xmin = xsteps[i]
      xmax = xsteps[i+1]
      ymax = ysteps[j]
      ymin = ysteps[j+1]
      # print("xmin = "+str(xmin))
      # print("xmax = "+str(xmax))
      # print("ymin = "+str(ymin))
      # print("ymax = "+str(ymax))
      # print("\n")
      gdal.Warp(output_path+str(i)+str(j)+".tif", dem,
            outputBounds=(xmin, ymin, xmax, ymax),
            dstNodata = no_data_value)
      # same result
      # gdal.Translate(dem_path+"/tiles/dem_tanslate"+str(i)+str(j)+".tif", dem,
      #                projWin = (xmin, ymax, xmax, ymin))


In [104]:
def create_complete_set(source_path, xdiv, ydiv, output_set_path):
  for i in range(len(xdiv)):
    dem = gdal.Open(source_path)
    band = dem.GetRasterBand(1)
    no_data_value = band.GetNoDataValue()
    gt = dem.GetGeoTransform()
    xmin = gt[0]
    ymax = gt[3]
    res = gt[1]
    xlen = res * dem.RasterXSize
    ylen = res * dem.RasterYSize
    xsize, ysize = get_tile_size(xlen=xlen, xdiv=xdiv[i], ylen=ylen, ydiv=ydiv[i])
    xsteps, ysteps = get_steps(xdiv[i], xmin, xsize, ydiv[i], ymax, ysize)
    set_path = output_set_path + "/" + str(xdiv[i]) + "x" + str(ydiv[i]) + " Tiles/"
    Path(set_path).mkdir(parents=True, exist_ok=True)
    tile(dem, set_path, xsteps, ysteps, no_data_value)
    dem = None

In [74]:
def get_files(folder, ext="tif"):
    """get all files with a specified extension in a folder"""
    glob_path = Path(folder)
    return [file for file in glob_path.glob(f"*.{ext}")]

In [90]:
dem = gdal.Open(high_dem_interpolated)
band = dem.GetRasterBand(1)
no_data_value = band.GetNoDataValue()

In [91]:
gt = dem.GetGeoTransform()
xmin = gt[0]
ymax = gt[3]
res = gt[1]
xlen = res * dem.RasterXSize
ylen = res * dem.RasterYSize

In [82]:
xdiv = 32
ydiv = 64

In [83]:
xsize, ysize = get_tile_size(xlen=xlen, xdiv=xdiv, ylen=ylen, ydiv=ydiv)

In [84]:
xsteps, ysteps = get_steps(xmin, xsize, ymax, ysize)

In [85]:
set_path = DEM_Tiles_Sets + "/" + str(xdiv) + "x" + str(ydiv) + " Tiles/"
Path(set_path).mkdir(parents=True, exist_ok=True)


In [86]:
tile(dem, set_path, xsteps, ysteps, no_data_value)

In [ ]:
fig, ax = plt.subplots(ydiv, xdiv, figsize=(10, 10))
plt.subplots_adjust(wspace =0.0001)
ax = ax.flatten('F')
# fig.tight_layout()
i=0
for file in get_files(set_path):
  with rasterio.open(file) as tile:
    ax[i].axis('off')
    rasterio.plot.show(tile, ax=ax[i])
    i+=1

In [88]:
print(xsize, ysize)

200.5625 177.171875


In [108]:
xdiv = [2, 4, 8, 16, 32]
ydiv = [4, 8, 16, 32, 64]

In [109]:
create_complete_set(annotation_file, xdiv, ydiv, Annotated_Tiles_Sets)